In [1]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd

Download and load a timeseries from the Yahoo dataset.

In [3]:
from mb_detect.dataloader.ds_loader import yahoo_data

#if mb_detect is not yet installed, uncomment the following lines instead of the import:
#def yahoo_data(file_path):
#    df = pd.read_csv(file_path)
#    df = df.rename(columns={"anomaly": "is_anomaly"})
#    df = df.astype({"is_anomaly": bool})
#    return df

In [17]:
# Insert the path to a csv file from the yahoo dataset:
yahoo_path = "/data/yahoo/ydata-labeled-time-series-anomalies-v1_0/A1Benchmark/real_67.csv"
df = yahoo_data(yahoo_path)

We create a connection to the database and insert and read the dataframe. This uses the connection the ManuBrain platform is delivered with. If no database is running, skip the next two cells. and uncomment the third.

In [ ]:
from sqlalchemy import create_engine

username = "opcuauser"
password = "opcuapassword123"
host = "opcua-processor-postgres-s:5432"
dbname = "opcuadb"

create_engine_str = "postgresql://" + username + ":" + password + "@" + host + "/" + dbname

engine = create_engine(create_engine_str)

In [ ]:
df.to_sql("yahoo_67", engine)
train_df = pd.read_sql("select * from yahoo_67", engine)

In [18]:
#train_df = df

This data can now immediatly be used by the detec

In [25]:
from mb_detect.models.classic import sklearn_dect
model = sklearn_dect.Models(model_type="forest", n_estimators=50).get_model()
X = train_df["value"].to_numpy().reshape(-1, 1)
pred_outliers = model.fit_predict(X)

In [28]:
result_df = train_df
result_df["pred_outliers"] = pred_outliers == -1
result_df

,timestamp,value,is_anomaly,pred_outliers
0,1,1,False,False
1,2,48,False,False
2,3,55,False,False
3,4,122,False,False
4,5,92,False,False
...,...,...,...,...
1418,1419,6742,True,True
1419,1420,8836,True,True
1420,1421,6509,True,True
1421,1422,9414,True,True
